In [1]:
from platform import python_version
print(python_version())


3.7.12


In [3]:
!python -m pip install shioaji

     |████████████████████████████████| 5.9 MB 4.3 MB/s 
     |████████████████████████████████| 54 kB 2.3 MB/s 
     |████████████████████████████████| 93 kB 1.6 MB/s 
     |████████████████████████████████| 243 kB 53.2 MB/s 
     |████████████████████████████████| 10.9 MB 32.9 MB/s 
     |████████████████████████████████| 18.1 MB 217 kB/s 
     |████████████████████████████████| 208 kB 56.5 MB/s 
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.1
    Uninstalling importlib-metadata-4.11.1:
      Successfully uninstalled importlib-metadata-4.11.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.

In [5]:
import shioaji as sj

In [6]:
# ---- Formal Login
api = sj.Shioaji()
accounts = api.login(
    "J120420374", 
    "celia5818"
)

api.activate_ca(
    #ca_path="C:/ekey/551/J120420374/S/Sinopac.pfx",
    ca_path="c:/eleader/ca/Sinopac.pfx",
    ca_passwd="J120420374",
    person_id="J120420374",
)


Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up


CaError: ignored

In [ ]:
# ---- Testing Login

#api = sj.Shioaji(backend='https', simulation=True)
api = sj.Shioaji(simulation=True)
accounts = api.login("PAPIUSER08", "2222")



In [ ]:
contract_dhf = api.Contracts.Futures.DHF['DHF202201'] #鴻海期貨202201 (=DHFA2)

In [ ]:
accounts = api.list_accounts()
print(accounts)


In [ ]:
# Stock default account
print(api.stock_account)

# Futures default account
print(api.futopt_account)


**粗體文字**--- Quote callback

In [ ]:
from shioaji import TickFOPv1, BidAskFOPv1, Exchange

@api.on_tick_fop_v1()
def quote_callback(exchange:Exchange, tick:TickFOPv1):
    print(f"Exchange: {exchange}, Tick: {tick}")
   
@api.on_bidask_fop_v1()
def quote_callback(exchange:Exchange, bidask:BidAskFOPv1):
    print(f"Exchange: {exchange}, BidAsk: {bidask}")


--- 訂閱盤中 tick & bidask(最佳五檔) 資料

In [ ]:

api.quote.subscribe(
    contract=contract_dhf, 
    quote_type="tick",
    version = sj.constant.QuoteVersion.v1, # or 'v1'
)

api.quote.subscribe(
    contract=contract_dhf, 
    quote_type = "bidask",
    version = sj.constant.QuoteVersion.v1 # or 'v1'
)

#print(contract_dhf)

In [ ]:
###### Unsubscribe tick data
api.quote.unsubscribe(contract_dhf, quote_type=sj.constant.QuoteType.Tick)
api.quote.unsubscribe(contract_dhf, quote_type=sj.constant.QuoteType.BidAsk)

--- Place Order


In [ ]:
# futures order
order1 = api.Order(
    action=sj.constant.Action.Buy,
    price=104.5,
    quantity=1,
    price_type=sj.constant.StockPriceType.LMT,
    order_type=sj.constant.FuturesOrderType.ROD, 
    octype=sj.constant.FuturesOCType.Auto,
    account=api.futopt_account
)

trade = api.place_order(contract_dhf, order1)
trade

# Cancel order
# api.cancel_order(trade)

# 修改order價格
# api.update_order(trade=trade, price=410)

# qty是指要減少的數量
# api.update_order(trade=trade, qty=1)

Trade(contract=Future(code='DHFA2', symbol='DHF202201', name='鴻海期貨01', category='DHF', delivery_month='202201', delivery_date='2022/01/19', underlying_kind='S', underlying_code='2317', unit=1, limit_up=116.5, limit_down=95.4, reference=106.0, update_date='2021/12/29'), order=Order(action=<Action.Buy: 'Buy'>, price=104.5, quantity=1, account=FutureAccount(person_id='J120420374', broker_id='F002000', account_id='1641626', username='黃瑞柳'), price_type=<StockPriceType.LMT: 'LMT'>, order_type=<FuturesOrderType.ROD: 'ROD'>), status=OrderStatus(status=<Status.Failed: 'Failed'>))

Response Code: 0 | Event Code: 12 | Info: solClient.c:12194                    (00001e5c) Session '(c0,s1)_sinopac' keep-alive on tcp_TxRx detected session down, client name 'PYAPI/J120420374/1229/025417/265181/114.137.81.245', VPN name 'sinopac', peer address 'hostname '203.66.91.161:80' IP 203. | Event: Session reconnecting
Session Down.
Response Code: 0 | Event Code: 1 | Info: solClientOS.c:4220                   (00001e5c) TCP connection failure for fd 1292, error = No route to host (10065) | Event: Session down


--- 接收報價，觸發訊號

In [ ]:
# 初始化 close series
import pandas as pd
import numpy as np
import time
from datetime import datetime

minute_close = pd.Series()
stock = 0

# 紀錄前12個close
for i in range(0,12):
    snapshots = api.snapshots(contracts)
    minute_close = minute_close.append(pd.Series(
        [snapshots[0].close], 
        index=[pd.to_datetime(snapshots[0].ts, unit='ns')]
    ))
    time.sleep(60)

# 開始算RSI
for i in range(0,700):
    # 抓snapshot
    snapshots = api.snapshots(contracts)
    
    # 存到分k收盤價的series
    minute_close = minute_close.append(pd.Series(
        [snapshots[0].close], 
        index=[pd.to_datetime(snapshots[0].ts, unit='ns')]
    ))
    
    # 計算rsi
    rsi = RSI(minute_close)

    # 觸發訊號判斷
    if rsi <= 30 and rsi >= 0 and stock == 0:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time, "BUY AT ", snapshots[0].close)
        stock += 1
    if rsi >= 70 and stock == 1:
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S")
        print("Current Time =", current_time, "SELL AT ", snapshots[0].close)
        stock -= 1
    time.sleep(60)

---檢視帳上庫存

In [ ]:
api.list_positions(api.stock_account)
